In [1]:
%cd ..

/Users/treycole/Codes/WanPy


In [2]:
%load_ext line_profiler
%timeit

In [3]:
from WanPy.WanPy import *
import WanPy.models as models
import WanPy.plotting as plot

from pythtb import *
import numba
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sympy as sp 
import scipy

In [4]:
delta = 1
t = -1
t2 = 0.2

model = models.Haldane(delta, t, t2).make_supercell([[2,0], [0,2]])

orbs = model.get_orb()
n_orb = model.get_num_orbitals()
n_occ = int(n_orb/2)
lat_vecs = model.get_lat() # lattice vectors

low_E_sites = np.arange(0, model.get_num_orbitals(), 2)
high_E_sites = np.arange(1, model.get_num_orbitals(), 2)

In [5]:
nkx = 10
nky = 10
Nk = nkx*nky

k_mesh = gen_k_mesh(nkx, nky, flat=False, endpoint=False)
k_mesh_flat = gen_k_mesh(nkx, nky, flat=True, endpoint=False)

u_wfs_Wan = wf_array(model, [nkx, nky])
for i in range(k_mesh.shape[0]):
    for j in range(k_mesh.shape[1]):
        u_wfs_Wan.solve_on_one_point(k_mesh[i,j], [i,j])
psi_wfs_Wan = get_bloch_wfs(orbs, u_wfs_Wan, k_mesh)

In [6]:
omit_sites = 4
tf_list = list(np.setdiff1d(low_E_sites, [omit_sites])) # delta on lower energy sites omitting the last site

W0, psi_til_wan = Wannierize(orbs, u_wfs_Wan, tf_list, ret_psi_til=True)
u_tilde_wan = get_bloch_wfs(orbs, psi_til_wan, k_mesh, inverse=True)
M = k_overlap_mat(lat_vecs, orbs, u_tilde_wan)  # [kx, ky, b, m, n]

In [7]:
spread, expc_rsq, expc_r_sq = spread_recip(lat_vecs, M, decomp=True)

print("After first projection")
print(rf"Spread = {spread[0]}")
print(rf"Omega_I = {spread[1]}")
print(rf"Omega_til = {spread[2]}")

After first projection
Spread = [0.22740033 0.22740033 0.22740033]
Omega_I = 0.527173883537877
Omega_til = 0.1550271029866458


In [8]:
# outer window of entangled bands is full occupied manifold
outer_states = u_wfs_Wan._wfs[..., :n_occ, :]

In [9]:
W0_max_loc, Wf_cntrs = max_loc_Wan(lat_vecs, orbs, u_wfs_Wan, tf_list, outer_states, 
        iter_num_omega_i=1, iter_num_omega_til=100,
        Wan_idxs=None, verbose=True, return_uwfs=False, return_wf_centers=True, eps=1e-3
        )

0 Omega_I: 0.5224352455932965
0 Omega_til = 0.14337782825095388, Grad mag: 4.67721090231174
1 Omega_til = 0.14327586978163412, Grad mag: 4.65149927060491
2 Omega_til = 0.14318106592846036, Grad mag: 4.625889595951855
3 Omega_til = 0.14309271275982274, Grad mag: 4.600383725743991
4 Omega_til = 0.14301018772512006, Grad mag: 4.574983340101565
5 Omega_til = 0.1429329394020536, Grad mag: 4.549689964847249
6 Omega_til = 0.14286047860386375, Grad mag: 4.524504983311828
7 Omega_til = 0.14279237066027178, Grad mag: 4.499429647095668
8 Omega_til = 0.14272822871130592, Grad mag: 4.4744650858947805
9 Omega_til = 0.1426677078764417, Grad mag: 4.449612316487058
10 Omega_til = 0.1426105001799689, Grad mag: 4.424872250963121
11 Omega_til = 0.1425563301308612, Grad mag: 4.4002457042764
12 Omega_til = 0.14250495086878764, Grad mag: 4.375733401178427
13 Omega_til = 0.14245614080068472, Grad mag: 4.3513359825980125
14 Omega_til = 0.14240970066255948, Grad mag: 4.327054011516352
15 Omega_til = 0.142365450

In [9]:
W0_max_loc, Wf_cntrs = max_loc_Wan(lat_vecs, orbs, u_wfs_Wan, tf_list, outer_states, 
        iter_num_omega_i=100, iter_num_omega_til=100,
        Wan_idxs=None, verbose=True, return_uwfs=False, return_wf_centers=True, eps=1e-3
        )

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
10 Omega_I: 0.48867378673602585
11 Omega_I: 0.48578198905465403
12 Omega_I: 0.48301327418404705
13 Omega_I: 0.4803737222134296
14 Omega_I: 0.47786681964443983
15 Omega_I: 0.475493746569461
16 Omega_I: 0.4732537429670067
17 Omega_I: 0.4711444912243334
18 Omega_I: 0.46916247631807456
19 Omega_I: 0.46730330365649536
20 Omega_I: 0.4655619672654045
21 Omega_I: 0.46393306868567136
22 Omega_I: 0.4624109910153313
23 Omega_I: 0.46099003422475665
24 Omega_I: 0.45966451817546417
25 Omega_I: 0.45842885934531563
26 Omega_I: 0.45727762651902565
27 Omega_I: 0.45620557987733384
28 Omega_I: 0.4552076971311633
29 Omega_I: 0.4542791896506486
30 Omega_I: 0.45341551094840576
31 Omega_I: 0.45261235938839023
32 Om

In [10]:
%lprun -f max_loc_Wan max_loc_Wan(lat_vecs, orbs, u_wfs_Wan, tf_list, outer_states, iter_num_omega_i=100, iter_num_omega_til=100, verbose=True, return_uwfs=False, return_wf_centers=True, eps=1e-3)

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
10 Omega_I: 0.48867378673602585
11 Omega_I: 0.48578198905465403
12 Omega_I: 0.48301327418404705
13 Omega_I: 0.4803737222134296
14 Omega_I: 0.47786681964443983
15 Omega_I: 0.475493746569461
16 Omega_I: 0.4732537429670067
17 Omega_I: 0.4711444912243334
18 Omega_I: 0.46916247631807456
19 Omega_I: 0.46730330365649536
20 Omega_I: 0.4655619672654045
21 Omega_I: 0.46393306868567136
22 Omega_I: 0.4624109910153313
23 Omega_I: 0.46099003422475665
24 Omega_I: 0.45966451817546417
25 Omega_I: 0.45842885934531563
26 Omega_I: 0.45727762651902565
27 Omega_I: 0.45620557987733384
28 Omega_I: 0.4552076971311633
29 Omega_I: 0.4542791896506486
30 Omega_I: 0.45341551094840576
31 Omega_I: 0.45261235938839023
32 Om

Timer unit: 1e-09 s

Total time: 1.65355 s
File: /Users/treycole/Codes/WanPy/WanPy/WanPy.py
Function: max_loc_Wan at line 864

Line #      Hits         Time  Per Hit   % Time  Line Contents
   864                                           def max_loc_Wan(
   865                                               lat_vecs,
   866                                               orbs,
   867                                               u_wfs,
   868                                               tf_list,
   869                                               outer_states,
   870                                               iter_num_omega_i=1000,
   871                                               iter_num_omega_til=1000,
   872                                               eps=1e-3,
   873                                               tol=1e-17,
   874                                               alpha=1,
   875                                               Wan_idxs=None,
   876                

In [10]:
%lprun -f max_loc_Wan max_loc_Wan(lat_vecs, orbs, u_wfs_Wan, tf_list, outer_states, iter_num_omega_i=100, iter_num_omega_til=100, verbose=True, return_uwfs=False, return_wf_centers=True, eps=1e-3)

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
10 Omega_I: 0.48867378673602585
11 Omega_I: 0.48578198905465403
12 Omega_I: 0.48301327418404705
13 Omega_I: 0.4803737222134296
14 Omega_I: 0.47786681964443983
15 Omega_I: 0.475493746569461
16 Omega_I: 0.4732537429670067
17 Omega_I: 0.4711444912243334
18 Omega_I: 0.46916247631807456
19 Omega_I: 0.46730330365649536
20 Omega_I: 0.4655619672654045
21 Omega_I: 0.46393306868567136
22 Omega_I: 0.4624109910153313
23 Omega_I: 0.46099003422475665
24 Omega_I: 0.45966451817546417
25 Omega_I: 0.45842885934531563
26 Omega_I: 0.45727762651902565
27 Omega_I: 0.45620557987733384
28 Omega_I: 0.4552076971311633
29 Omega_I: 0.4542791896506486
30 Omega_I: 0.45341551094840576
31 Omega_I: 0.45261235938839023
32 Om

Timer unit: 1e-09 s

Total time: 1.16343 s
File: /Users/treycole/Codes/WanPy/WanPy/WanPy.py
Function: max_loc_Wan at line 891

Line #      Hits         Time  Per Hit   % Time  Line Contents
   891                                           def max_loc_Wan(
   892                                               lat_vecs,
   893                                               orbs,
   894                                               u_wfs,
   895                                               tf_list,
   896                                               outer_states,
   897                                               iter_num_omega_i=1000,
   898                                               iter_num_omega_til=1000,
   899                                               eps=1e-3,
   900                                               tol=1e-17,
   901                                               alpha=1,
   902                                               Wan_idxs=None,
   903                

In [16]:
from scipy.sparse.linalg import expm_multiply

In [ ]:
expm

In [11]:
%lprun -f find_min_unitary find_min_unitary(lat_vecs, M, iter_num=100, eps=1e-3, verbose=True, tol=1e-17)

0 Omega_til = 0.15491402050243544, Grad mag: 4.726324345017292
1 Omega_til = 0.15480909872956716, Grad mag: 4.700296710623302
2 Omega_til = 0.15471152306816785, Grad mag: 4.6743730249579025
3 Omega_til = 0.15462057378786062, Grad mag: 4.6485551397037685
4 Omega_til = 0.15453561402777638, Grad mag: 4.622844737455427
5 Omega_til = 0.1544560793920921, Grad mag: 4.597243344872234
6 Omega_til = 0.15438146892202428, Grad mag: 4.571752344646321
7 Omega_til = 0.1543113372555918, Grad mag: 4.546372986410806
8 Omega_til = 0.15424528781343536, Grad mag: 4.5211063966986735
9 Omega_til = 0.15418296687107264, Grad mag: 4.495953588049235
10 Omega_til = 0.15412405839789048, Grad mag: 4.470915467347904
11 Omega_til = 0.15406827955937186, Grad mag: 4.445992843474945
12 Omega_til = 0.15401537679377503, Grad mag: 4.421186434330243
13 Omega_til = 0.15396512238656693, Grad mag: 4.396496873293582
14 Omega_til = 0.15391731147639431, Grad mag: 4.371924715173378
15 Omega_til = 0.15387175943563747, Grad mag: 4.3

Timer unit: 1e-09 s

Total time: 0.609914 s
File: /Users/treycole/Codes/WanPy/WanPy/WanPy.py
Function: find_min_unitary at line 754

Line #      Hits         Time  Per Hit   % Time  Line Contents
   754                                           def find_min_unitary(lat_vecs, M, eps=1 / 160, iter_num=10, verbose=False, tol=1e-17):
   755                                               """
   756                                               Finds the unitary that minimizing the gauge dependent part of the spread. 
   757                                           
   758                                               Args:
   759                                                   lat_vecs: Lattice vectors
   760                                                   M: Overlap matrix
   761                                                   eps: Step size for gradient descent
   762                                                   iter_num: Number of iterations
   763                             

In [22]:
%lprun -f find_optimal_subspace find_optimal_subspace(lat_vecs, orbs, outer_states, u_tilde_wan, iter_num=100, verbose=True)

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
10 Omega_I: 0.48867378673602585
11 Omega_I: 0.48578198905465403
12 Omega_I: 0.48301327418404705
13 Omega_I: 0.4803737222134296
14 Omega_I: 0.47786681964443983
15 Omega_I: 0.475493746569461
16 Omega_I: 0.4732537429670067
17 Omega_I: 0.4711444912243334
18 Omega_I: 0.46916247631807456
19 Omega_I: 0.46730330365649536
20 Omega_I: 0.4655619672654045
21 Omega_I: 0.46393306868567136
22 Omega_I: 0.4624109910153313
23 Omega_I: 0.46099003422475665
24 Omega_I: 0.45966451817546417
25 Omega_I: 0.45842885934531563
26 Omega_I: 0.45727762651902565
27 Omega_I: 0.45620557987733384
28 Omega_I: 0.4552076971311633
29 Omega_I: 0.4542791896506486
30 Omega_I: 0.45341551094840576
31 Omega_I: 0.45261235938839023
32 Om

Timer unit: 1e-09 s

Total time: 1.07317 s
File: /Users/treycole/Codes/WanPy/WanPy/WanPy.py
Function: find_optimal_subspace at line 643

Line #      Hits         Time  Per Hit   % Time  Line Contents
   643                                           def find_optimal_subspace(
   644                                               lat_vecs, orbs, outer_states, inner_states, iter_num=100, verbose=False, tol=1e-17, alpha=1
   645                                           ):
   646         1       1000.0   1000.0      0.0      if isinstance(inner_states, wf_array):
   647                                                   shape = inner_states._wfs.shape  # [*nks, idx, orb]
   648                                                   inner_states = np.array(inner_states._wfs)
   649                                               else:
   650         1       1000.0   1000.0      0.0          shape = inner_states.shape  # [*nks, idx, orb]
   651                                           
   652       

In [14]:
%timeit max_loc_Wan(lat_vecs, orbs, u_wfs_Wan, tf_list, outer_states, iter_num_omega_i=10, iter_num_omega_til=10, verbose=True, return_uwfs=False, return_wf_centers=True, eps=1e-3)

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
0 Omega_til = 0.08442524880031389, Grad mag: 4.269932415992194
1 Omega_til = 0.08435062730041262, Grad mag: 4.24618202706687
2 Omega_til = 0.08428082813245769, Grad mag: 4.222541773171438
3 Omega_til = 0.0842153995117566, Grad mag: 4.19901231525619
4 Omega_til = 0.08415394054407369, Grad mag: 4.175594229850294
5 Omega_til = 0.08409609490275863, Grad mag: 4.152288015711397
6 Omega_til = 0.08404154533837359, Grad mag: 4.129094099868098
7 Omega_til = 0.08399000890721399, Grad mag: 4.106012843120941
8 Omega_til = 0.08394123282067412, Grad mag: 4.083044545059464
9 Omega_til = 0.08389499083127766, Grad mag: 4.060189448645748
Post processing report:
 --------------- 
Quadratic spread = [0.19185829 

In [12]:
%timeit get_max_loc_uwfs(lat_vecs, orbs, u_tilde_wan, eps=1e-3, iter_num=10, verbose=True, tol=1e-17)

0 Omega_til = 0.15491402050243544, Grad mag: 4.726324345017292
1 Omega_til = 0.15480909872956716, Grad mag: 4.700296710623302
2 Omega_til = 0.15471152306816785, Grad mag: 4.6743730249579025
3 Omega_til = 0.15462057378786062, Grad mag: 4.6485551397037685
4 Omega_til = 0.15453561402777638, Grad mag: 4.622844737455427
5 Omega_til = 0.1544560793920921, Grad mag: 4.597243344872234
6 Omega_til = 0.15438146892202428, Grad mag: 4.571752344646321
7 Omega_til = 0.1543113372555918, Grad mag: 4.546372986410806
8 Omega_til = 0.15424528781343536, Grad mag: 4.5211063966986735
9 Omega_til = 0.15418296687107264, Grad mag: 4.495953588049235
0 Omega_til = 0.15491402050243544, Grad mag: 4.726324345017292
1 Omega_til = 0.15480909872956716, Grad mag: 4.700296710623302
2 Omega_til = 0.15471152306816785, Grad mag: 4.6743730249579025
3 Omega_til = 0.15462057378786062, Grad mag: 4.6485551397037685
4 Omega_til = 0.15453561402777638, Grad mag: 4.622844737455427
5 Omega_til = 0.1544560793920921, Grad mag: 4.597243

In [16]:
%timeit find_min_unitary(lat_vecs, M, iter_num=10, eps=1e-3, verbose=True, tol=1e-17)

0 Omega_til = 0.15491402050243544, Grad mag: 4.726324345017292
1 Omega_til = 0.15480909872956716, Grad mag: 4.700296710623302
2 Omega_til = 0.15471152306816785, Grad mag: 4.6743730249579025
3 Omega_til = 0.15462057378786062, Grad mag: 4.6485551397037685
4 Omega_til = 0.15453561402777638, Grad mag: 4.622844737455427
5 Omega_til = 0.1544560793920921, Grad mag: 4.597243344872234
6 Omega_til = 0.15438146892202428, Grad mag: 4.571752344646321
7 Omega_til = 0.1543113372555918, Grad mag: 4.546372986410806
8 Omega_til = 0.15424528781343536, Grad mag: 4.5211063966986735
9 Omega_til = 0.15418296687107264, Grad mag: 4.495953588049235
0 Omega_til = 0.15491402050243544, Grad mag: 4.726324345017292
1 Omega_til = 0.15480909872956716, Grad mag: 4.700296710623302
2 Omega_til = 0.15471152306816785, Grad mag: 4.6743730249579025
3 Omega_til = 0.15462057378786062, Grad mag: 4.6485551397037685
4 Omega_til = 0.15453561402777638, Grad mag: 4.622844737455427
5 Omega_til = 0.1544560793920921, Grad mag: 4.597243

In [16]:
%timeit find_optimal_subspace(lat_vecs, orbs, outer_states, u_tilde_wan, iter_num=10, verbose=True)

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: